> Eğer `OPENAI_API_KEY` tanımlı değilse, terminale aşağıdakini yazın:  
> `export OPENAI_API_KEY="sk-..."`  
> ya da `.env` dosyasına bu satırı ekleyin ve Jupyter'ı yeniden başlatın.

In [190]:
# 01_settings_check    | OPENAI_API_KEY, pandas, openai sürüm çıktısı

# LIBS & BASIC SETUP
import os, json, time, sys, subprocess
from datetime import datetime

# 1 | Paketi mevcut değilse kur → sadece bir kez çalışır
for pkg in ("pandas", "openai"):
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

import pandas as pd
import openai

print("Pandas :", pd.__version__, "| OpenAI :", openai.__version__)



# 2 | API Key
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise EnvironmentError("OPENAI_API_KEY bulunamadı → terminalde `export OPENAI_API_KEY=...`")

# 3 | Dosya kontrol
DATA_PATH = "cleaned_result.json"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"'{DATA_PATH}' dosyası yok – çalışma dizini: {os.getcwd()}")

with open(DATA_PATH, encoding="utf-8") as f:
    raw = json.load(f)
df = pd.DataFrame(raw)
print("Veri yüklendi | Kayıt :", len(df))
df.head(2)


Pandas : 2.3.1 | OpenAI : 1.97.1
Veri yüklendi | Kayıt : 3000


,İş ID,Lokasyon,Sektör,İş Gönderim Saati,İade Talebi Saati,Hizmet Veren Geri Arama Saatleri,Hizmet Veren Arama Metinleri,Partner Geri Arama Saatleri,Partner Arama Metinleri,İade Durumu,İade Talebi Nedeni,Partner Red Sebebi,Müşteri Cevabı,Sistem Cevabı
0,115607,Şanlıurfa Haliliye,Buzdolabı Servisi,2025-07-22 16:00:02,2025-07-22 16:32:59,2025-07-22 16:30:46,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Servisnucreti kabul. Etmiyor iade istiyorum,None,None,None
1,115605,İstanbul Pendik,Bulaşık Makinesi Servisi,2025-07-22 15:58:56,2025-07-22 17:27:46,2025-07-22 17:26:54,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Müşteri dün kendisi halletmiş su kapalıymış su...,None,None,İş gönderimi ile müşterinin aranma saati arası...


In [191]:
# 02_data_cleaning    | JSON okuma, NaN düzeltme, datetime, duplicate

# CLEANING & FEATURES
date_cols = ["İş Gönderim Saati", "İade Talebi Saati"]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce")

text_cols = [
    "İade Talebi Nedeni","Hizmet Veren Arama Metinleri","Partner Arama Metinleri",
    "Partner Red Sebebi","Müşteri Cevabı","Sistem Cevabı","Lokasyon","Sektör"
]
for c in text_cols:
    df[c] = df[c].fillna("").astype(str)

# Süre (dk)
df["Time_Elapsed_Minutes"] = (
    (df["İade Talebi Saati"] - df["İş Gönderim Saati"]).dt.total_seconds() / 60
)

# Hedef etiket → iki değer
df = df[df["İade Durumu"].isin(["Personel Kabul Etti","Personel Redetti"])].copy()
print("Kayıt (etiketli) :", len(df))


Kayıt (etiketli) : 2531


In [192]:
# 03_feature_call_counts   | arama sayısı hesapla → hizmet veren ve partner

def count_calls(value):
    """
    Hizmet veren veya partner tarafından yapılan arama sayısını hesaplar.
    '||' ile ayrılmış metin segmentlerinin sayısına göre döner.
    """
    if pd.isna(value) or value == "":
        return 0
    return len(str(value).split("||"))

# Yeni sayısal sütunları oluştur
df["HizmetVerenAramaSayisi"] = df["Hizmet Veren Arama Metinleri"].apply(count_calls)
df["PartnerAramaSayisi"] = df["Partner Arama Metinleri"].apply(count_calls)

# Bilgi ver ve genel istatistikleri göster
print("Arama sayıları eklendi.")
df[["HizmetVerenAramaSayisi", "PartnerAramaSayisi"]].describe()


Arama sayıları eklendi.


,HizmetVerenAramaSayisi,PartnerAramaSayisi
count,2531.000000,2531.000000
mean,1.882655,0.835243
std,1.547936,1.298874
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,2.000000,1.000000
max,24.000000,17.000000


In [193]:
# 04_filter_and_prompt | system/user/assistant mesajı + JSONL oluştur

import json, pathlib

def default(val, placeholder="(veri yok)"):
    return val if val and str(val).strip() else placeholder

SYSTEM_PROMPT = (
    "You are an AI assistant representing the internal personnel of a service platform. "
    "Your role is to review refund requests submitted by service providers and make the final decision: "
    "'Accepted by Personnel' or 'Rejected by Personnel'. "
    "You must base your decision on the following structured information:\n\n"
    "- Reason for Refund Request: A short written justification from the provider.\n"
    "- Service Provider Call Transcripts: Transcript(s) of provider's calls with the customer.\n"
    "- Service Provider Call Count: How many times the provider attempted to reach the customer (based on call segments).\n"
    "- Partner Call Transcripts: Transcript(s) of the partner's confirmation calls with the customer.\n"
    "- Partner Call Count: How many times the partner called the customer for confirmation (based on call segments).\n"
    "- Time Elapsed Between Job Submission and Refund Request: Time in minutes.\n\n"
    "Analyze whether the provider gives a valid, timely, and reasonable explanation. "
    "Reject vague, insufficient, abusive claims, or cases where fault lies with the provider. "
    "Accept cases with clear external causes (e.g., customer unavailable, location inaccessible, technical issues beyond provider control). "
    "Do not assume anything beyond the given data.\n\n"
    "Return explanation in Turkish. Cevap açıklaması kısa ve Türkçe olmalıdır.\n\n"
    "Respond ONLY with a JSON object:\n"
    "{'decision': 'Accepted by Personnel' or 'Rejected by Personnel', 'reason': 'short explanation in Turkish'}"
)

out = []

for _, row in df.iterrows():
    user_parts = [
        f"Reason for Refund Request: {default(row['İade Talebi Nedeni'])}",
        f"Service Provider Call Transcripts: {default(row['Hizmet Veren Arama Metinleri'])}",
        f"Service Provider Call Count: {default(row['HizmetVerenAramaSayisi'])}",
        f"Partner Call Transcripts: {default(row['Partner Arama Metinleri'])}",
        f"Partner Call Count: {default(row['PartnerAramaSayisi'])}",
        f"Time Elapsed Between Job Submission and Refund Request: "
        f"{'' if pd.isna(row['Time_Elapsed_Minutes']) else int(row['Time_Elapsed_Minutes'])} minutes."
    ]
    user_msg = "\n".join(p for p in user_parts if p.strip())

    decision = (
        "Accepted by Personnel"
        if row["İade Durumu"] == "Personel Kabul Etti"
        else "Rejected by Personnel"
    )

    # GPT-4o-mini için bağlamsal reason oluşturma
    raw_reason = str(row.get("İade Talebi Nedeni") or "")
    partner_note = row.get("Partner Arama Metinleri") or ""
    sistem_yanit = row.get("Sistem Cevabı") or ""
    partner_red = row.get("Partner Red Sebebi") or ""

    reason_parts = []

    if "müşteri vazgeçti" in raw_reason.lower():
        reason_parts.append("Müşteri vazgeçtiği belirtildi.")
    elif "ulaşılamıyor" in raw_reason.lower():
        reason_parts.append("Müşteriye ulaşılamadığı bildirildi.")
    elif "fiyat" in raw_reason.lower() or "ücret" in raw_reason.lower():
        reason_parts.append("Müşteri fiyatı kabul etmedi.")
    elif "parça yok" in raw_reason.lower() or "yedek parça" in raw_reason.lower():
        reason_parts.append("Yedek parça bulunamadı.")

    if partner_note:
        reason_parts.append("Partner görüşmesi değerlendirildi.")
    if sistem_yanit:
        reason_parts.append("Sistem cevabı incelendi.")
    if partner_red:
        reason_parts.append("Partner red gerekçesi dikkate alındı.")

    reason_str = " ".join(reason_parts).strip()
    if not reason_str:
        reason_str = "İade talebi metni değerlendirildi."
    if len(reason_str) > 100:
        reason_str = reason_str[:97] + "..."

    assistant_msg = json.dumps(
        {"decision": decision, "reason": reason_str},
        ensure_ascii=False,
        separators=(",", ": ")
    )

    out.append(json.dumps(
        {"messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]},
        ensure_ascii=False
    ))

JSONL_PATH = "fine_tuning_data.jsonl"
pathlib.Path(JSONL_PATH).write_text("\n".join(out), encoding="utf-8")
print("JSONL yazıldı →", JSONL_PATH, "| Satır:", len(out))


JSONL yazıldı → fine_tuning_data.jsonl | Satır: 2531


In [194]:
# 05_fine_tune_start.ipynb | JSONL dosyasını yükle, fine-tuning job’u başlat

# UPLOAD + FINE-TUNE
import openai

try:
    with open(JSONL_PATH, "rb") as f:
        file_id = openai.files.create(file=f, purpose="fine-tune").id
    print("File ID:", file_id)

    job = openai.fine_tuning.jobs.create(
        training_file=file_id,
        model="gpt-4o-mini-2024-07-18"  # MODEL GÜNCELLENDİ
    )
    job_id = job.id
    print("Job ID:", job_id)

except openai.BadRequestError as e:
    if "exceeded_quota" in str(e):
        print("Uyarı: API key içinde yeterli bakiye yok. Lütfen sistem hesabıyla yeniden deneyin.")
    else:
        raise e


File ID: file-9aLCHFJmEsTWHBoUTiMfJa
Uyarı: API key içinde yeterli bakiye yok. Lütfen sistem hesabıyla yeniden deneyin.


In [195]:
# 06_cost_estimation   | Toplam örnek ve token sayısı hesapla

import sys
import subprocess
import json
from pathlib import Path

# tiktoken yüklü mü? Değilse yükle
try:
    import tiktoken
except ImportError:
    print("'tiktoken' modülü eksik. Yükleniyor...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tiktoken"])
    import tiktoken

# Model adı net olarak snapshot ile eşleşsin
encoding = tiktoken.encoding_for_model("gpt-4o-mini-2024-07-18")

jsonl_path = Path("fine_tuning_data.jsonl")

# Sayıcılar
total_tokens = 0
total_examples = 0

# Dosya satır satır okunur
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        messages = data["messages"]
        msg_text = "".join(m["content"] for m in messages)
        tokens = len(encoding.encode(msg_text))
        total_tokens += tokens
        total_examples += 1

# Sonuç
print("Toplam örnek sayısı:", total_examples)
print("Toplam token sayısı:", total_tokens)


Toplam örnek sayısı: 2531
Toplam token sayısı: 2009972


In [197]:
# 07_follow_job | Fine-tuning job durumunu izle ve model adını kaydet

import time, sys
from datetime import datetime

while True:
    status = openai.fine_tuning.jobs.retrieve(job_id).status
    print(datetime.now().strftime("%H:%M:%S"), "→", status)
    if status in ("succeeded", "failed"):
        break
    time.sleep(15)

if status == "succeeded":
    model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
    print("Model hazır →", model_name)

    # MODEL ADINI KAYDET → test hücresinde doğrudan kullanılabilsin
    with open("fine_tuned_model.txt", "w") as f:
        f.write(model_name)
    print("Model adı 'fine_tuned_model.txt' dosyasına yazıldı.")

else:
    print("Eğitim başarısız:", job_id)


NotFoundError: Error code: 404 - {'error': {'message': 'Could not find fine tune: dummy_job_id', 'type': 'invalid_request_error', 'param': 'fine_tune_id', 'code': 'fine_tune_not_found'}}

In [198]:
# 08_model_test | Fine-tuned modele örnek prompt gönder ve yanıtı al

import openai, json
from datetime import datetime

# 1. Sistem mesajını yeniden tanımla (tutarlılık için)
SYSTEM_PROMPT = (
    "You are an AI assistant representing the internal personnel of a service platform. "
    "Your role is to review refund requests submitted by service providers and make the final decision: "
    "'Accepted by Personnel' or 'Rejected by Personnel'. "
    "You must base your decision on the following structured information:\n\n"
    "- Reason for Refund Request: A short written justification from the provider.\n"
    "- Service Provider Call Transcripts: Transcript(s) of provider's calls with the customer.\n"
    "- Service Provider Call Count: How many times the provider attempted to reach the customer (based on call segments).\n"
    "- Partner Call Transcripts: Transcript(s) of the partner's confirmation calls with the customer.\n"
    "- Partner Call Count: How many times the partner called the customer for confirmation (based on call segments).\n"
    "- Time Elapsed Between Job Submission and Refund Request: Time in minutes.\n\n"
    "Analyze whether the provider gives a valid, timely, and reasonable explanation. "
    "Reject vague, insufficient, abusive claims, or cases where fault lies with the provider. "
    "Accept cases with clear external causes (e.g., customer unavailable, location inaccessible, technical issues beyond provider control). "
    "Do not assume anything beyond the given data.\n\n"
    "Return explanation in Turkish. Cevap açıklaması kısa ve Türkçe olmalıdır.\n\n"
    "Respond ONLY with a JSON object:\n"
    "{'decision': 'Accepted by Personnel' or 'Rejected by Personnel', 'reason': 'short explanation in Turkish'}"
)

# 2. Model adını txt dosyasından al
with open("fine_tuned_model.txt") as f:
    final_model_name = f.read().strip()
print("Model kullanılacak:", final_model_name)

# 3. GPT-4o-mini’ye test prompt gönder
def evaluate(model_name: str,
            reason: str,
            provider_transcripts: str = "",
            partner_transcripts: str = "",
            provider_call_count: int = 0,
            partner_call_count: int = 0,
            elapsed_minutes: int = 0) -> dict:
    
    user_prompt = (
        f"Reason for Refund Request: {reason}\n"
        f"Service Provider Call Transcripts: {provider_transcripts}\n"
        f"Service Provider Call Count: {provider_call_count}\n"
        f"Partner Call Transcripts: {partner_transcripts}\n"
        f"Partner Call Count: {partner_call_count}\n"
        f"Time Elapsed Between Job Submission and Refund Request: {elapsed_minutes} minutes."
    )

    response = openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    # Assistant cevabını JSON olarak parse et
    return json.loads(response.choices[0].message.content)

# 4. Örnek test verisi
result = evaluate(
    model_name=final_model_name,
    reason="Müşteri yanlış adres vermiş, hizmet iptal edilmiş.",
    provider_transcripts="Müşteri adresi Zeytinburnu demişti ama aradığım kişi Erzurum'daydı.",
    partner_transcripts="Partner: Müşteri adres bilgisinin yanlış olduğunu doğruladı.",
    provider_call_count=2,
    partner_call_count=1,
    elapsed_minutes=22
)

# 5. Sonuç yazdır
print("Decision:", result["decision"])
print("Reason  :", result["reason"])


FileNotFoundError: [Errno 2] No such file or directory: 'fine_tuned_model.txt'

In [199]:
# 09_eval_set_analysis | Fine-tuned modelin doğruluğunu örnek eval set ile test et

import random

# 1. Eval set: rastgele 10 kayıt
eval_sample = df.sample(n=10, random_state=42).copy()

# 2. Model tahmini ve karşılaştırma
eval_results = []
for _, row in eval_sample.iterrows():
    expected = (
        "Accepted by Personnel"
        if row["İade Durumu"] == "Personel Kabul Etti"
        else "Rejected by Personnel"
    )

    response = evaluate(
        final_model_name,
        reason=row["İade Talebi Nedeni"],
        provider_transcripts=row["Hizmet Veren Arama Metinleri"],
        partner_transcripts=row["Partner Arama Metinleri"],
        provider_call_count=row["HizmetVerenAramaSayisi"],
        partner_call_count=row["PartnerAramaSayisi"],
        elapsed_minutes=0 if pd.isna(row["Time_Elapsed_Minutes"]) else int(row["Time_Elapsed_Minutes"])
    )

    predicted = response.get("decision", "").strip()
    reason = response.get("reason", "").strip()

    eval_results.append({
        "expected_decision": expected,
        "predicted_decision": predicted,
        "reason": reason,
        "correct": expected.lower() == predicted.lower()
    })

# 3. Accuracy hesapla ve yazdır
correct_count = sum(1 for r in eval_results if r["correct"])
total = len(eval_results)
accuracy = correct_count / total
print(f"\nEval Set Accuracy: {accuracy:.2%} ({correct_count}/{total})")

# 4. Hatalı tahminleri detaylı yazdır
print("\nHatalı Tahminler:")
for r in eval_results:
    if not r["correct"]:
        print(f"Expected: {r['expected_decision']} | Predicted: {r['predicted_decision']}")
        print(f"📝 Reason: {r['reason']}")
        print("-" * 60)


NameError: name 'final_model_name' is not defined

In [200]:
# 10_eval_metrics | Accuracy, Precision, Recall, F1-score hesapla

# scikit-learn yüklü değilse otomatik yükle
try:
    from sklearn.metrics import classification_report
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn"])
    from sklearn.metrics import classification_report

# Normalize edilmiş sınıf etiketleri (case-insensitive karşılaştırmalarla tutarlı olur)
expected = [r["expected_decision"].strip().lower() for r in eval_results]
predicted = [r["predicted_decision"].strip().lower() for r in eval_results]

# Boşluk kontrolü
if not expected or not predicted:
    print("Uyarı: Değerlendirme listeleri boş. Eval set henüz oluşturulmamış olabilir.")
else:
    print("Sınıflandırma Raporu:\n")
    print(classification_report(expected, predicted, digits=3, labels=["accepted by personnel", "rejected by personnel"]))


Uyarı: Değerlendirme listeleri boş. Eval set henüz oluşturulmamış olabilir.


In [201]:
# 11_error_analysis | Hatalı tahminlerde model gerekçesini detaylı analiz et

print("Hatalı Tahminler - Detaylı İnceleme:\n")

for i, row in enumerate(eval_results):
    if not row["correct"]:
        print(f"#{i+1}")
        print("Beklenen Karar :", row["expected_decision"])
        print("Model Tahmini  :", row["predicted_decision"])
        print("Model Gerekçesi:", row.get("reason", "(Açıklama bulunamadı)"))
        print("-" * 70)


Hatalı Tahminler - Detaylı İnceleme:



In [202]:
# 12_export_eval | Eval sonuçlarını CSV dosyasına dışa aktar

import pandas as pd

# DataFrame oluştur
eval_df = pd.DataFrame(eval_results)

# Sütun sırasını elle belirle (görsel temizlik)
columns = [
    "expected_decision",
    "predicted_decision",
    "correct",
    "reason"
]
# Sadece bu sütunları al, varsa diğerlerini de bırak
eval_df = eval_df[[col for col in columns if col in eval_df.columns]]

# CSV dosyasına yaz
eval_df.to_csv("eval_results.csv", index=False, encoding="utf-8-sig")

print("Eval sonuçları CSV dosyasına yazıldı → eval_results.csv")


Eval sonuçları CSV dosyasına yazıldı → eval_results.csv
